# PART 2A - DIMENSION REDUCTION
Topic modeling using LDA and NMF<br>
For the final model I have used LDA <br><br>
LDA -> reads the term frequency files - dill pipeline, train and test TF vector<br>
Writes -> lda pipeline, LDA topic model output vector <br><br>
NMF -> reads the term frequency inverse document frequency files - dill pipeline, train and test TFID vector<br>
Writed  -> nmf pipeline, NMF topic model output vector <br>

This notebook can be run but LDA , NMF takes about 4 mins each.

In [2]:
from __future__ import print_function
# key libs
import numpy as np
import re
import nltk
import pandas as pd
import glob
import codecs

# nlp libs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 

# processing
from sklearn.model_selection import train_test_split

# LDA
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF


# bring in my pickled vectorizers
import pickle
import dill
# import import_ipynb
# from process_tokenize import my_tokenizer

In [3]:
# what are we interestd in
# set this up to read the correct set of files we are interested in

n_gram = '1gm' # options - 1gm , 2gm
stem_type = 'lemma' # options - lemma,snow

In [4]:
# set random state
RSEED = 0

### 1. OPEN - TERM FREQUENCY FILES 
i.e. output of CountVectorizer

In [5]:
tf_train = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_train.csv', compression = 'gzip')
tf_train = tf_train.drop(columns='Unnamed: 0')

tf_test = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tf_test = tf_test.drop(columns='Unnamed: 0')

In [6]:
tf_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0,0,2,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,3,0,0


In [7]:
# load the appropriate tf_vectorizer
tf_vectorizer = dill.load(open('../data/vectors/tf_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### 2. OPEN - TERM FREQUENCY INVERSE DOCUMENT FREQUENCY FILES
i.e output of TfidfVectorizer 

In [11]:
tfid_train = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_train.csv',compression = 'gzip')
tfid_train = tfid_train.drop(columns='Unnamed: 0')

tfid_test = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tfid_test = tfid_test.drop(columns='Unnamed: 0')

In [12]:
tfid_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.000000
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.0,0.0,0.004126,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.005299
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.0,0.0,0.006395,0.002774,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00448,0.00000,0.0,0.000000
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.005872,...,0.0,0.0,0.0,0.0,0.0,0.00554,0.00000,0.00000,0.0,0.000000
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.0,0.0,0.000000,0.013355,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.03942,0.0,0.000000


In [27]:
# load the appropriate tf_vectorizer
tfid_vectorizer = dill.load(open('../data/vectors/tfid_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### 3. HELPER FUNCTIONS

In [8]:
#function to display topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print (f'Topic: {topic_idx}')
        print (" , ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [9]:
#function to display books for a topic
def find_topic_books(df,number):
    topics = list(df.columns)[3:]
    #print(topics)
    for t in topics:
        temp = df.sort_values(by=t,ascending=False)
        print('\n------'+t+'-------') 
        print(temp.iloc[0:number+1,0:2])    

### 4. DIMENSIONALITY REDUCTION - TOPIC MODELLING
1. LDA
2. NMF
3. LSA / LSI ? - These are rather old and don't give good results


use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well

In [10]:
# set how many topics you want
topic_number = 20

### A. LDA with TF
LatentDirichletAllocation with the Term Frequency Allocation
Use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well
LDA can only use raw term counts for LDA because it is a probabilistic graphical model

In [11]:
# Run LDA 
lda_tf = LatentDirichletAllocation(learning_method= 'batch',n_components=topic_number, random_state=RSEED, batch_size=128,n_jobs=-1)
lda_tf.fit(tf_train.iloc[:,3:])

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
# transform vectors
vec_lda_train = lda_tf.transform(tf_train.iloc[:,3:])
vec_lda_test = lda_tf.transform(tf_test.iloc[:,3:])

In [13]:
dill.dump(lda_tf, open('../data/vectors/lda_'+ stem_type + '_' + n_gram,'wb'))

In [14]:
print('------PERPLEXITY------')
print(lda_tf.perplexity(tf_train.iloc[:,3:]))
print(lda_tf.perplexity(tf_test.iloc[:,3:]))

------PERPLEXITY------
4939.035264152425
5214.084644529548


In [15]:
# get the topics and words for the topics in lda
no_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
display_topics(lda_tf, tf_feature_names, no_top_words)

Topic: 0
thou , king , thee , thy , sir , lord , lady , father , answered , queen
Topic: 1
sir , lord , letter , lady , person , manner , family , father , gentleman , character
Topic: 2
girl , don , sat , horse , replied , answered , rose , jimmy , strange , suddenly
Topic: 3
sea , boat , ship , captain , wind , island , tree , land , big , deck
Topic: 4
thy , thou , poem , poet , soul , nature , thee , line , earth , spirit
Topic: 5
king , ship , father , town , prince , army , english , sent , england , horse
Topic: 6
sir , lady , gentleman , dear , cried , answered , miss , table , madame , window
Topic: 7
tom , dick , boy , don , sam , answered , cried , yes , fellow , didn
Topic: 8
don , mr , miss , yes , sir , boy , didn , ain , girl , won
Topic: 9
girl , boy , mother , don , peter , tree , cried , story , father , dear
Topic: 10
law , class , money , general , person , power , fact , society , public , trade
Topic: 11
government , general , power , law , war , king , public , l

In [16]:
# add back details
lda_train = pd.DataFrame(vec_lda_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_train.insert(loc=0, column='book_location', value=tf_train['book_location'])
lda_train.insert(loc=0, column='author_name', value=tf_train['author_name'])
lda_train.insert(loc=0, column='book_title', value=tf_train['book_title'])
lda_train.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_train.csv')
lda_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.000006,0.000006,0.000006,0.030122,0.002298,0.000006,0.156122,...,0.024089,0.000006,0.000006,0.000006,0.000006,0.125936,0.000006,0.102446,0.000006,0.160367
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.000003,0.026168,0.000003,0.000003,0.000003,0.000003,0.036873,...,0.003894,0.000003,0.000003,0.000003,0.004125,0.051193,0.181417,0.000003,0.000003,0.132154
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.000353,0.023734,0.066106,0.107061,0.000004,0.048938,0.205313,...,0.008678,0.008994,0.006023,0.037082,0.000004,0.103316,0.053795,0.000004,0.000004,0.156578
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.000009,0.000009,0.433507,0.084543,0.051103,0.000009,0.023939,...,0.000009,0.000009,0.067776,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.002889
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.000009,0.033941,0.003979,0.064638,0.000009,0.000009,0.519437,...,0.015295,0.012108,0.015163,0.008986,0.064570,0.017562,0.017422,0.025508,0.012847,0.100762


In [17]:
# add the book details and write to file for test
lda_test = pd.DataFrame(vec_lda_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_test.insert(loc=0, column='book_location', value=tf_test['book_location'])
lda_test.insert(loc=0, column='author_name', value=tf_test['author_name'])
lda_test.insert(loc=0, column='book_title', value=tf_test['book_title'])
lda_test.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_test.csv')
lda_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.198687,0.000012,0.000012,0.000012,0.000012,0.000012,0.238428,...,0.000012,0.000012,0.000012,0.055709,0.000012,0.203518,0.121411,0.000794,0.000012,0.000012
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.000003,0.000003,0.000003,0.022477,0.000003,0.000003,0.002076,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.063450,0.000003,0.000003,0.000003
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.000004,0.000004,0.000004,0.000004,0.000004,0.133392,0.064586,...,0.020249,0.000004,0.000004,0.000004,0.000004,0.000004,0.083705,0.000004,0.000004,0.000004
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.000025,0.000025,0.000025,0.000025,0.000025,0.024634,0.000025,...,0.040010,0.048756,0.000025,0.000025,0.558395,0.293079,0.000025,0.000025,0.012130,0.000025
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.000403,0.000002,0.081530,0.030420,0.008572,0.000002,0.080594,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.005832,0.000002,0.003153,0.454447


In [18]:
# find documents for topic
no_documents = 10
find_topic_books(lda_train,no_documents)


------topic_1-------
                                         book_title          author_name
2020  The Story of the Champions of the Round Table          Howard Pyle
180                                 Comic Tragedies    Louisa May Alcott
668                                       Cleopatra  Henry Rider Haggard
1707                             Idylls of the King        Lord Tennyson
738                                 Eric Brighteyes  Henry Rider Haggard
1183             The Merry Adventures of Robin Hood          Howard Pyle
2140                            The Last Tournament        Lord Tennyson
265                                    Morning Star  Henry Rider Haggard
263                                    The Brethren  Henry Rider Haggard
256                                  Moon of Israel  Henry Rider Haggard
1170                          Queen Mary and Harold        Lord Tennyson

------topic_2-------
                           book_title                author_name
120            

In [19]:
# setup for visualization
vec_tf=csr_matrix(tf_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(lda_tf,vec_tf,tf_vectorizer)

/Users/user/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.016316 -0.000124       1        1  8.911619
19     0.026575 -0.013019       2        1  8.091551
6      0.034040  0.021683       3        1  7.833238
3      0.135990 -0.021425       4        1  5.983681
15    -0.149475  0.013309       5        1  5.343936
8      0.122955 -0.046262       6        1  5.212588
1     -0.079966  0.016364       7        1  5.143738
12     0.050379 -0.038726       8        1  5.005476
9      0.168134  0.033118       9        1  4.893344
2      0.106509 -0.042680      10        1  4.766829
4     -0.032038  0.161350      11        1  4.765189
0      0.086500  0.165045      12        1  4.685106
14    -0.180695 -0.106173      13        1  4.243616
13     0.032051 -0.124363      14        1  4.242605
17    -0.085963  0.018742      15        1  4.181643
11    -0.198796 -0.008137      16        1  4.171154
5      0.027265 -0.007839      17        1  3.777041
10    -0.196434 -0.083571      18        1  3.128264
18    -0.053186  0.173645      19        1  3.082546
7      0.169839 -0.110936      20        1  2.536835, topic_info=      Category           Freq        Term          Total  loglift  logprob
term                                                                      
7167   Default   82022.000000         god   82022.000000  30.0000  30.0000
4937   Default  151256.000000         don  151256.000000  29.0000  29.0000
10562  Default  120142.000000          mr  120142.000000  28.0000  28.0000
9218   Default   75503.000000        king   75503.000000  27.0000  27.0000
16409  Default   45596.000000        thou   45596.000000  26.0000  26.0000
1882   Default   83991.000000         boy   83991.000000  25.0000  25.0000
9296   Default  118696.000000        lady  118696.000000  24.0000  24.0000
16585  Default   26866.000000         tom   26866.000000  23.0000  23.0000
14768  Default  119438.000000         sir  119438.000000  22.0000  22.0000
4484   Default   20492.000000        dick   20492.000000  21.0000  21.0000
16482  Default   34971.000000         thy   34971.000000  20.0000  20.0000
9732   Default   80643.000000        lord   80643.000000  19.0000  19.0000
10351  Default   78036.000000        miss   78036.000000  18.0000  18.0000
16341  Default   29392.000000        thee   29392.000000  17.0000  17.0000
7076   Default   79960.000000        girl   79960.000000  16.0000  16.0000
2301   Default   57365.000000     captain   57365.000000  15.0000  15.0000
14553  Default   40401.000000        ship   40401.000000  14.0000  14.0000
6991   Default   59782.000000     general   59782.000000  13.0000  13.0000
656    Default   58760.000000    answered   58760.000000  12.0000  12.0000
15191  Default   18944.000000      specie   18944.000000  11.0000  11.0000
4496   Default   37492.000000        didn   37492.000000  10.0000  10.0000
3750   Default   51102.000000       cried   51102.000000   9.0000   9.0000
18397  Default   77609.000000         yes   77609.000000   8.0000   8.0000
1763   Default   38527.000000        boat   38527.000000   7.0000   7.0000
6168   Default  112161.000000      father  112161.000000   6.0000   6.0000
14264  Default   56363.000000         sea   56363.000000   5.0000   5.0000
9398   Default   48010.000000         law   48010.000000   4.0000   4.0000
8437   Default   25010.000000      indian   25010.000000   3.0000   3.0000
11056  Default   27988.000000     officer   27988.000000   2.0000   2.0000
7211   Default   24123.000000  government   24123.000000   1.0000   1.0000
...        ...            ...         ...            ...      ...      ...
9063   Topic20    4339.393613         jim   10401.092772   2.8001  -5.6301
13805  Topic20    2303.197071       roger    5450.342657   2.8129  -6.2636
12814  Topic20    1634.547745  questioned    4044.951130   2.7681  -6.6065
16820  Topic20    1903.883016        trip    4974.877892   2.7137  -6.4540
10091  Topic20  

### B. NMF WITH TFID
NMF workd better with TFID

In [84]:
# Run NMF
nmf_tfid = NMF(n_components=topic_number, random_state=RSEED, max_iter=500)
nmf_tfid.fit(tfid_train.iloc[:,3:])

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=500,
  n_components=20, random_state=0, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [85]:
# transform vectors
vec_nmf_train = nmf_tfid.transform(tfid_train.iloc[:,3:])
vec_nmf_test = nmf_tfid.transform(tfid_test.iloc[:,3:])

In [86]:
dill.dump(nmf_tfid, open('../data/vectors/nmf_'+ stem_type + '_' + n_gram,'wb'))

In [87]:
no_top_words = 10
tfid_feature_names = tfid_vectorizer.get_feature_names()
display_topics(nmf_tfid, tfid_feature_names, no_top_words)

Topic: 0
answered , cried , madame , replied , monsieur , lip , girl , exclaimed , smile , continued
Topic: 1
thou , thy , thee , hath , ti , thine , hast , heaven , soul , art
Topic: 2
art , poet , public , character , literature , english , government , modern , society , author
Topic: 3
mr , husband , didn , couldn , wouldn , isn , won , doesn , wasn , girl
Topic: 4
sir , lord , gentleman , john , george , duke , harry , master , london , james
Topic: 5
s , wot , arter , ave , ginger , sam , bob , agin , ead , skipper
Topic: 6
captain , ship , boat , deck , island , vessel , board , crew , sail , mate
Topic: 7
dick , tom , sam , rover , baxter , dan , fred , cried , answered , cadet
Topic: 8
green , bird , hill , flower , sky , wood , mountain , sweet , blue , dream
Topic: 9
god , christ , lord , sin , jesus , soul , heaven , christian , holy , church
Topic: 10
specie , animal , darwin , plant , hypothesis , science , organic , natural , modification , bird
Topic: 11
officer , army 

In [88]:
# add back details
nmf_train = pd.DataFrame(vec_nmf_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_train.insert(loc=0, column='book_location', value=tfid_train['book_location'])
nmf_train.insert(loc=0, column='author_name', value=tfid_train['author_name'])
nmf_train.insert(loc=0, column='book_title', value=tfid_train['book_title'])
nmf_train.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_train.csv')
nmf_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.024754,0.0,0.030954,0.032591,0.081775,0.000000,0.000000,...,0.000000,0.000000,0.072496,0.000000,0.043274,0.000000,0.005502,0.000000,0.053511,0.000891
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.010176,0.0,0.015237,0.047607,0.006928,0.001250,0.000000,...,0.000000,0.000000,0.003213,0.000000,0.000000,0.000000,0.241563,0.077373,0.007175,0.004169
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.044107,0.0,0.014085,0.000000,0.083100,0.000000,0.013311,...,0.003250,0.007783,0.030975,0.003627,0.005625,0.012087,0.013309,0.053859,0.000000,0.019906
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.004341,0.0,0.000000,0.022038,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.043628,0.000000,0.000330,0.040634,0.000000,0.000000,0.000000,0.000000
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.063080,0.0,0.018698,0.000363,0.090716,0.001838,0.010200,...,0.025048,0.003055,0.004595,0.000000,0.004226,0.008518,0.000547,0.056877,0.000000,0.006891


In [89]:
# add back details
nmf_test = pd.DataFrame(vec_nmf_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_test.insert(loc=0, column='book_location', value=tfid_test['book_location'])
nmf_test.insert(loc=0, column='author_name', value=tfid_test['author_name'])
nmf_test.insert(loc=0, column='book_title', value=tfid_test['book_title'])
nmf_test.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_test.csv')
nmf_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.013377,0.001834,0.010979,0.002782,0.006235,0.000320,0.011939,...,0.000000,0.009849,0.001425,0.005960,0.000819,0.0,0.003740,0.011713,0.060561,0.000653
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.011489,0.000000,0.000000,0.053352,0.003168,0.394411,0.006154,...,0.000000,0.000000,0.008906,0.000000,0.013697,0.0,0.004149,0.037200,0.000000,0.000000
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.008674,0.000000,0.000000,0.018326,0.047545,0.000000,0.000000,...,0.000000,0.000000,0.034548,0.039140,0.000000,0.0,0.000000,0.000000,0.000000,0.000247
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.003790,0.000260,0.023789,0.000000,0.000000,0.000000,0.002772,...,0.063213,0.005000,0.000000,0.006096,0.000000,0.0,0.000000,0.000000,0.000000,0.002766
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.095430,0.000000,0.000000,0.034680,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.049041,0.000000,0.001582,0.0,0.004010,0.057867,0.000000,0.000000


In [90]:
# find documents for topic
no_documents = 30
find_topic_books(nmf_train,no_documents)


------topic_1-------
                                             book_title  \
254                                       The Missioner   
550                                    The Great Secret   
1037                                        The Avenger   
1854                                  The Yellow Crayon   
2127                             The Vanished Messenger   
2179                               Mysterious Mr. Sabin   
984                                   The Master Mummer   
564                                            Berenice   
1319                                              Havoc   
1672                        The Trampling of the Lilies   
791                                   Mr. Marx's Secret   
1826                                         After Dark   
1180                                     The Malefactor   
1235                                St. Martin's Summer   
1919                                    Poor Miss Finch   
1508                              

In [91]:
# setup for visualization
vec_tfid=csr_matrix(tfid_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(nmf_tfid,vec_tfid,tfid_vectorizer)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      12.747954        1       1 -0.217977 -0.082265
0       8.084530        1       2 -0.027522 -0.088157
4       7.843532        1       3 -0.085834 -0.043873
8       6.874480        1       4  0.015135  0.131293
15      6.614661        1       5 -0.020437 -0.001372
12      6.190448        1       6  0.138922 -0.070451
17      5.724203        1       7  0.029689 -0.092775
11      5.613333        1       8 -0.137938 -0.039769
3       5.518599        1       9  0.067304 -0.186479
13      5.153137        1      10 -0.090067  0.167443
9       4.972857        1      11 -0.160590  0.153528
10      4.581926        1      12 -0.167089 -0.099130
1       4.543971        1      13 -0.092724  0.235424
6       4.499612        1      14 -0.005772 -0.017890
18      3.134909        1      15 -0.125556 -0.037543
7       2.131263        1      16  0.179619 -0.012514
16      1.836390        1      17  0.242580  0.065428
14      1.798166        1      18  0.176993  0.156110
19      1.392345        1      19  0.010779 -0.113427
5       0.743683        1      20  0.270485 -0.023581, topic_info=      Category        Freq          Term       Total  loglift  logprob
term                                                                  
10481  Default  217.000000            mr  217.000000  30.0000  30.0000
4455   Default   93.000000          dick   93.000000  29.0000  29.0000
13849  Default   69.000000             s   69.000000  28.0000  28.0000
7110   Default  123.000000           god  123.000000  27.0000  27.0000
8090   Default   73.000000  illustration   73.000000  26.0000  26.0000
9156   Default  119.000000          king  119.000000  25.0000  25.0000
11694  Default   72.000000         peter   72.000000  24.0000  24.0000
16456  Default   75.000000           tom   75.000000  23.0000  23.0000
16281  Default   82.000000          thou   82.000000  22.0000  22.0000
10273  Default   96.000000          miss   96.000000  21.0000  21.0000
13915  Default   60.000000           sam   60.000000  20.0000  20.0000
16353  Default   71.000000           thy   71.000000  19.0000  19.0000
7822   Default   39.000000        holmes   39.000000  18.0000  18.0000
16214  Default   57.000000          thee   57.000000  17.0000  17.0000
9002   Default   38.000000         jimmy   38.000000  16.0000  16.0000
14656  Default  100.000000           sir  100.000000  15.0000  15.0000
9658   Default   89.000000          lord   89.000000  14.0000  14.0000
2283   Default   69.000000       captain   69.000000  13.0000  13.0000
13766  Default   34.000000         rover   34.000000  12.0000  12.0000
12316  Default   55.000000        prince   55.000000  11.0000  11.0000
18131  Default   25.000000           wot   25.000000  10.0000  10.0000
2628   Default   38.000000        christ   38.000000   9.0000   9.0000
4467   Default   59.000000          didn   59.000000   8.0000   8.0000
14443  Default   48.000000          ship   48.000000   7.0000   7.0000
7021   Default   77.000000          girl   77.000000   6.0000   6.0000
1094   Default   40.000000          aunt   40.000000   5.0000   5.0000
1596   Default   24.000000         billy   24.000000   4.0000   4.0000
6978   Default   41.000000        george   41.000000   3.0000   3.0000
1133   Default   19.000000           ave   19.000000   2.0000   2.0000
1752   Default   46.000000          boat   46.000000   1.0000   1.0000
...        ...         ...           ...         ...      ...      ...
820    Topic20    2.211350           ard    4.212092   4.2570  -5.3812
3255   Topic20    1.370778      conjurer    2.815935   4.1814  -5.8594
12038  Topic20    2.740771          pore    5.635389   4.1805  -5.1666
11120  Topic20    2.040539          ouse    4.217434   4.1753  -5.4616
8991   Topic20    0.919531         jetty    1.972628   4.1381  -6.2587
13834  Topic20    1.244858        russet    2.701664   4.1265  -5.9558


In [ ]:
### next -> featurize_cluster.ipynb